# **EMOTION DETECTION**

In [1]:
# Getting all dependencies

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import pickle


In [2]:
# Getting files on Collab
# uploaded = files.upload()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Specify the name of the uploaded zip file
# zip_file_name = 'images.zip'

# Unzip the file
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    # zip_ref.extractall('')

# Verify the contents
# print(os.listdir('images'))

In [4]:
# Setting up the directories

TRAINING_DIR = './images/train'
TEST_DIR = './images/validation'

In [5]:
# Making dataframes for the training and test data (with images and labels)

def create_dataframe(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image_paths.append(image_path)
                labels.append(label)
            print(f"Completed {label_dir}")
    return image_paths, labels

In [6]:
# Making Train and Test dataframes
train_df = pd.DataFrame()
train_df['image_path'], train_df['label'] = create_dataframe(TRAINING_DIR)

test_df = pd.DataFrame()
test_df['image_path'], test_df['label'] = create_dataframe(TEST_DIR)
print("Train and test dataframes created successfully.")

print(train_df)
print(test_df)

Completed ./images/train\angry
Completed ./images/train\disgust
Completed ./images/train\fear
Completed ./images/train\happy
Completed ./images/train\neutral
Completed ./images/train\sad
Completed ./images/train\surprise
Completed ./images/validation\angry
Completed ./images/validation\disgust
Completed ./images/validation\fear
Completed ./images/validation\happy
Completed ./images/validation\neutral
Completed ./images/validation\sad
Completed ./images/validation\surprise
Train and test dataframes created successfully.
                             image_path     label
0            ./images/train\angry\0.jpg     angry
1            ./images/train\angry\1.jpg     angry
2           ./images/train\angry\10.jpg     angry
3        ./images/train\angry\10002.jpg     angry
4        ./images/train\angry\10016.jpg     angry
...                                 ...       ...
28816  ./images/train\surprise\9969.jpg  surprise
28817  ./images/train\surprise\9985.jpg  surprise
28818  ./images/train\sur

In [7]:
# Label Encoding

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
y_train = label_encoder.transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

In [34]:
# Image transforms

transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])



In [35]:
# Custom Dataset class

class EmotionDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('L')
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [36]:
# Create datasets and dataloaders

train_dataset = EmotionDataset(train_df['image_path'].tolist(), y_train, transform=transform)
test_dataset = EmotionDataset(test_df['image_path'].tolist(), y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [40]:
# Convolutional Neural Network (CNN) model
# The model is a sequential model with convolutional layers, max pooling layers, and dropout layers
# The model is designed to classify images into 7 different classes (emotions)

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.4)

        self.fc1 = nn.Linear(512 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout(x)

        x = x.view(-1, 512 * 3 * 3)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.fc5(x)
        return x

In [ ]:
# Configuring the model

model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [42]:
# Training loop with validation
for epoch in range(150):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/150 [Train]"):
        inputs, labels = inputs.to(device), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(test_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f"Epoch {epoch+1}/150 - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")


Epoch 1/150 [Train]: 100%|██████████| 226/226 [00:17<00:00, 12.68it/s]


Epoch 1/150 - Train Loss: 1.8636, Train Acc: 21.31% | Val Loss: 1.8242, Val Acc: 25.83%


Epoch 2/150 [Train]: 100%|██████████| 226/226 [00:18<00:00, 12.30it/s]


Epoch 2/150 - Train Loss: 1.8279, Train Acc: 23.09% | Val Loss: 1.8025, Val Acc: 26.04%


Epoch 3/150 [Train]: 100%|██████████| 226/226 [00:15<00:00, 14.93it/s]


...

Epoch 148/150 - Train Loss: 0.7926, Train Acc: 71.28% | Val Loss: 1.0251, Val Acc: 64.90%


Epoch 149/150 [Train]: 100%|██████████| 226/226 [00:15<00:00, 14.69it/s]


Epoch 149/150 - Train Loss: 0.7857, Train Acc: 71.39% | Val Loss: 1.0247, Val Acc: 65.48%


Epoch 150/150 [Train]: 100%|██████████| 226/226 [00:15<00:00, 14.40it/s]


Epoch 150/150 - Train Loss: 0.7914, Train Acc: 70.98% | Val Loss: 1.0239, Val Acc: 65.00%


In [44]:
# Save model
torch.save(model.state_dict(), "emotionDetector.pth")
print("Model saved successfully.")

Model saved successfully.


In [ ]:
# For Collab
# Download the JSON file
# files.download("emotionDetector.json")

# Download the HDF5 file
# files.download("emotionDetector.h5")